# Structural Estimation HW4

### Takuya Ando

Import packages

In [53]:
# Import packages and load the data
import numpy as np
import pandas as pd
import numpy.random as rnd
import numpy.linalg as lin
import scipy.stats as sts
import scipy.integrate as intgr
import scipy.optimize as opt
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
cmap1 = matplotlib.cm.get_cmap('summer')
# This next command is specifically for Jupyter Notebook
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

(a)Load data

In [54]:
macro = pd.read_csv("data/NewMacroSeries.txt", header=None)
macro.columns = ["c_t", "k_t", "w_t", "r_t", "y_t"]
macro.head()

,c_t,k_t,w_t,r_t,y_t
0,1.128323e+07,8.040697e+06,1.120211e+07,1.008852,1.931398e+07
1,1.215464e+07,8.030754e+06,1.206726e+07,1.088112,2.080561e+07
2,1.097303e+07,8.650974e+06,1.089414e+07,0.911904,1.878300e+07
3,9.711635e+06,7.809971e+06,9.641815e+06,0.893986,1.662382e+07
4,9.245673e+06,6.912184e+06,9.179203e+06,0.961637,1.582621e+07


First, we define function that draws N x S values from a normal distribution.

In [55]:
def norm_draws(unif_vals, mu, sigma):
    '''
    This fundtion draws T x S values from a normal distribution
    based on uniform distribution
    '''
    norm_draws = sts.norm.ppf(unif_vals, loc=0, scale=sigma)
    
    return norm_draws

Then, we define another function which generate simulation values for z, k, w, r, c and y.

In [56]:
def sim_draw(unif_vals, k, alpha, beta, mu, sigma, p, T, S):
    '''
    This function draws simulation values for z, k, w, r, c and y.
    '''
    draws = norm_draws(unif_vals, mu, sigma)
    
    # z simulation draw
    z_sim = np.zeros((T,S))

    z_sim[0,:] = p * mu + (1-p) * mu + draws[0, :]

    for i in range(1, len(z_sim)):
        z_sim[i,:] = p * z_sim[i-1,:] + (1-p) * mu + draws[i,:]
    
    # k simulation draw
    k_sim = np.zeros((T+1, S))

    k_sim[0] = np.mean(k)

    for i in range(1, T+1):
        k_sim[i,:]=alpha*beta*np.exp(z_sim[i-1,:])*((k_sim[i-1,:])**alpha)
    
    # w, r simulation draw
    w_sim = np.zeros((T, S))

    r_sim = np.zeros((T, S))

    for i in range(T):
        w_sim[i, :] = (1-alpha)*np.exp(z_sim[i,:])*((k_sim[i,:])**alpha)
        r_sim[i, :] = alpha*np.exp(z_sim[i,:])*((k_sim[i,:])**(alpha-1))
    
    # c simulation draw
    c_sim = np.zeros((T, S))

    for i in range(T):
        c_sim[i,:] = w_sim[i, :] + np.multiply(r_sim[i, :], k_sim[i, :]) - k_sim[i+1, :]
    
    # y simulation draw
    y_sim = np.zeros((T, S))
   
    for i in range(T):
        y_sim[i, :]=np.exp(z_sim[i,:])*((k_sim[i,:])**alpha)
    
    
    k_sim = k_sim[0:100, :]
    
    return z_sim, k_sim, w_sim, r_sim, c_sim, y_sim

Then, we degine moment function both for data and model. 

In [57]:
def moments(k, w, r, c, y, data=True):
    '''
    This function compute moments for observed data 
    and simulation values(moments vector of length S)
    '''
    if data:
        mean_c = np.mean(c)
        mean_k = np.mean(k)
        mean_c_y = np.mean(np.divide(c, y))
        var_y = np.var(y)
        c_t = c[1:100]
        c_t_1 = c[0:99]
        corr_c = np.corrcoef(c_t, c_t_1)[1, 0]
        corr_ck = np.corrcoef(c, k)[1, 0]
    else:
        mean_c = np.mean(c, axis=0)
        mean_k = np.mean(k, axis=0)
        mean_c_y = np.mean(np.divide(c, y), axis=0)
        var_y = np.var(y, axis=0)
        c_t = c[1:100, :]
        c_t_1 = c[0:99, :]
        corr_c = np.zeros(1000)
        for i in range(1000):
            corr_c[i] = np.corrcoef(c_t[:,i], c_t_1[:,i])[1, 0]
        corr_ck = np.zeros(1000)
        for i in range(1000):
            corr_ck[i] = np.corrcoef(c[:,i], k[:,i])[1, 0]
    
    return mean_c, mean_k, mean_c_y, var_y, corr_c, corr_ck

Also, we define error vector and criterion functions.

In [58]:
def err_vec(data_vals, unif_vals, alpha, beta, mu, sigma, p, T, S, simple):
    '''
    --------------------------------------------------------------------
    This function computes the vector of moment errors (in percent
    deviation from the data moment vector) for SMM.
    --------------------------------------------------------------------
    '''
    k, w, r, c, y = data_vals
    
    z_sim, k_sim, w_sim, r_sim, c_sim, y_sim = \
    sim_draw(unif_vals, k, alpha, beta, mu, sigma, p, T, S)
    
    mom_data1, mom_data2, mom_data3, mom_data4,\
    mom_data5 ,mom_data6, = moments(k, w, r, c, y, data=True)
    
    moms_data = np.array([mom_data1, mom_data2, mom_data3, mom_data4, mom_data5, mom_data6])
    
    mom_sim1, mom_sim2, mom_sim3, mom_sim4,\
    mom_sim5 ,mom_sim6, = moments(k_sim, w_sim, r_sim, c_sim, y_sim, data=False)
    
    moms_model = np.array([np.mean(mom_sim1), np.mean(mom_sim2), np.mean(mom_sim3),\
                           np.mean(mom_sim4), np.mean(mom_sim5), np.mean(mom_sim6)])
    
    if simple:
        err_vec = moms_model - moms_data
    else:
        err_vec = (moms_model - moms_data) / moms_data
    
    return err_vec


def criterion(params, *args):
    '''
    --------------------------------------------------------------------
    This function computes the SMM weighted sum of squared moment errors
    criterion function value given parameter values and an estimate of
    the weighting matrix.
    --------------------------------------------------------------------
    '''
    alpha, mu, sigma, p = params
    beta=0.99
    datavals, unif_vals, T, S, W_hat = args
    err = err_vec(datavals, unif_vals, alpha, beta, mu, sigma, p, T, S, simple=False)
    crit_val = err.T @ W_hat @ err
    
    return crit_val

Now we perform SMM estimation. The estimated parametes are alpha_SMM1= 0.42105186489272567, mu_SMM1= 9.935073825212754, sig_SMM1= 0.08846794502048456 and p_SMM1= 0.9204782651500025. The minimized criterion function value is 4.485252339611445e-06.

In [59]:
# Extract observed data
k = macro.k_t
w = macro.w_t
r = macro.r_t
c = macro.c_t
y = macro.y_t

# Set T and S
T = 100
S = 1000

# Set initial parameters
alpha_init = 0.5

z = np.log(r/(alpha_init*(k**(alpha_init-1))))

mu_init = np.mean(z)

sigma_init = np.sqrt(np.var(z))

p_init = 0.01

params_init1 = np.array([alpha_init, mu_init, sigma_init, p_init])

datavals = (k, w, r, c, y)

# Generate uniform distribution
np.random.seed(1)
unif_vals = sts.uniform.rvs(0, 1, size=(T, S)) 

W_hat1 = np.eye(6)
smm_args1 = (datavals, unif_vals, T, S, W_hat1)
results1 = opt.minimize(criterion, params_init1, args=(smm_args1),
                          method='L-BFGS-B',
                          bounds=((1e-10, 1-1e-10), (1e-10, None), (1e-10, None),(-1+1e-10, 1-1e-10)))
alpha_SMM1, mu_SMM1, sig_SMM1, p_SMM1 = results1.x
print('alpha_SMM1=', alpha_SMM1, ' mu_SMM1=', mu_SMM1, ' sig_SMM1=', sig_SMM1, ' p_SMM1=', p_SMM1)
results1

alpha_SMM1= 0.42105186489272567  mu_SMM1= 9.935073825212754  sig_SMM1= 0.08846794502048456  p_SMM1= 0.9204782651500025


      fun: 4.485252339611445e-06
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-2.95966572e-04, -1.31317897e-05, -1.74179734e-05, -1.49612368e-05])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 295
      nit: 42
   status: 0
  success: True
        x: array([0.42105186, 9.93507383, 0.08846795, 0.92047827])

Also, vector of moment difference at the optimmum is as below.

In [68]:
err_vec(datavals, unif_vals, alpha_SMM1, 0.99, mu_SMM1, sig_SMM1, p_SMM1, T, S, False)

array([ 7.54546152e-04, -7.65937633e-04, -1.78251668e-03, -3.49684770e-07,
        2.74872686e-04, -2.76280946e-04])

Then, we will examine the standard error of each estimation. Here we define Jacobian matrix function.

In [60]:
def Jac_err(data_vals, unif_vals, alpha, mu, sigma, p, simple=False):
    '''
    This function computes the Jacobian matrix of partial derivatives of the R x 1 moment
    error vector e(x|theta) with respect to the K parameters theta_i in the K x 1 parameter vector
    theta. The resulting matrix is R x K Jacobian.
    '''
    Jac_err = np.zeros((6, 4))
    h_alpha = 1e-4 * alpha
    h_mu = 1e-4 * mu
    h_sig = 1e-4 * sigma
    h_p = 1e-4 * p
    Jac_err[:, 0] = \
        ((err_vec(data_vals, unif_vals, alpha + h_alpha, 0.99, mu, sigma, p,T,S, simple) -
          err_vec(data_vals, unif_vals, alpha - h_alpha, 0.99, mu, sigma, p,T,S, simple)) / (2 * h_alpha)).flatten()
    
    Jac_err[:, 1] = \
        ((err_vec(data_vals, unif_vals, alpha, 0.99, mu + h_mu, sigma, p,T,S, simple) -
          err_vec(data_vals, unif_vals, alpha, 0.99, mu - h_mu, sigma, p,T,S, simple)) / (2 * h_mu)).flatten()
    
    Jac_err[:, 2] = \
        ((err_vec(data_vals, unif_vals, alpha, 0.99, mu, sigma + h_sig, p,T,S, simple) -
          err_vec(data_vals, unif_vals, alpha, 0.99, mu, sigma - h_sig, p,T,S, simple)) / (2 * h_sig)).flatten()
    
    Jac_err[:, 3] = \
        ((err_vec(data_vals, unif_vals, alpha, 0.99, mu, sigma, p + h_p,T,S, simple) -
          err_vec(data_vals, unif_vals, alpha, 0.99, mu, sigma, p - h_p,T,S, simple)) / (2 * h_p)).flatten()
    
    
    return Jac_err

Using Jacobian matrix, we calculate variance covariance matrix for estimated parameters. Below are Jacobian matrix, variance covariance matrix and standard errors for estimated parameters.

In [61]:
d_err1 = Jac_err(datavals, unif_vals, alpha_SMM1, mu_SMM1, sig_SMM1, p_SMM1, False)
print(d_err1)
SigHat1 = (1 / S) * lin.inv(d_err1.T @ W_hat1 @ d_err1)
print(SigHat1)
print('Std. err. alpha_hat=', np.sqrt(SigHat1[0, 0]))
print('Std. err. mu_hat=', np.sqrt(SigHat1[1, 1]))
print('Std. err. sig_hat=', np.sqrt(SigHat1[2, 2]))
print('Std. err. p_hat=', np.sqrt(SigHat1[3, 3]))

[[ 2.70583090e+01  1.71639333e+00  1.41467162e+00  7.27242023e-01]
 [ 3.07732830e+01  1.69651151e+00  1.39682340e+00  7.18879220e-01]
 [-1.69462513e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 6.12934420e+01  3.45799097e+00  2.71843717e+01  1.18921732e+01]
 [ 1.28677827e-01  8.44926888e-05 -8.82548804e-02  4.25360501e-01]
 [ 2.03204601e-01  4.14372768e-03 -8.60920976e-02  4.31000793e-01]]
[[ 9.02516162e-05 -1.51843610e-03  1.67231785e-06 -2.74587936e-05]
 [-1.51843610e-03  2.57747430e-02  1.58600670e-05  2.92261561e-04]
 [ 1.67231785e-06  1.58600670e-05  4.28096020e-04 -9.88645142e-04]
 [-2.74587936e-05  2.92261561e-04 -9.88645142e-04  2.31715017e-03]]
Std. err. alpha_hat= 0.009500085064042714
Std. err. mu_hat= 0.1605451431783314
Std. err. sig_hat= 0.020690481376158337
Std. err. p_hat= 0.048136786063251585


(b)First, we construct the error matrix function.

In [62]:
def get_Err_mat(datavals, unif_vals, alpha, beta, mu, sigma, p, simple=False):
    '''
    --------------------------------------------------------------------
    This function computes the R x S matrix of errors from each
    simulated moment for each moment error. In this function, we have
    hard coded R = 6.
    --------------------------------------------------------------------
    '''
    R = 6
    S = 1000
    T = 100
    Err_mat = np.zeros((R, S))
    k, w, r, c, y = datavals
    
    z_sim, k_sim, w_sim, r_sim, c_sim, y_sim = \
    sim_draw(unif_vals, k, alpha, beta, mu, sigma, p, T, S)
    
    mom_data1, mom_data2, mom_data3, mom_data4,\
    mom_data5 ,mom_data6, = moments(k, w, r, c, y, data=True)
    
    mom_sim1, mom_sim2, mom_sim3, mom_sim4,\
    mom_sim5 ,mom_sim6, = moments(k_sim, w_sim, r_sim, c_sim, y_sim, data=False)
    
    if simple:
        Err_mat[0, :] = mom_sim1 - mom_data1
        Err_mat[1, :] = mom_sim2 - mom_data2
        Err_mat[2, :] = mom_sim3 - mom_data3
        Err_mat[3, :] = mom_sim4 - mom_data4
        Err_mat[4, :] = mom_sim5 - mom_data5
        Err_mat[5, :] = mom_sim6 - mom_data6
        
        
    else:
        Err_mat[0, :] = (mom_sim1 - mom_data1) / mom_data1
        Err_mat[1, :] = (mom_sim2 - mom_data2) / mom_data2
        Err_mat[2, :] = (mom_sim3 - mom_data3) / mom_data3
        Err_mat[3, :] = (mom_sim4 - mom_data4) / mom_data4
        Err_mat[4, :] = (mom_sim5 - mom_data5) / mom_data5
        Err_mat[5, :] = (mom_sim6 - mom_data6) / mom_data6
    
    return Err_mat

Then, we will construct optimal weighting matrix using the parameters from (a). the optimal weighting matrix is as follows.

In [63]:
Err_mat = get_Err_mat(datavals, unif_vals, alpha_SMM1, 0.99, mu_SMM1, sig_SMM1, p_SMM1, False)
VCV = (1 / S) * (Err_mat @ Err_mat.T)
print(VCV)
# Because VCV4 is poorly conditioned we use the pseudo-inverse to invert it, which
# uses the SVD
W_hat2 = lin.pinv(VCV)
print(W_hat2.shape)

[[ 2.96467480e-02  2.94244849e-02 -1.34499110e-06  7.76814009e-02
   7.51482709e-04  7.58574521e-04]
 [ 2.94244849e-02  2.92199398e-02  1.36529661e-06  7.71700387e-02
   7.44525977e-04  7.52563748e-04]
 [-1.34499110e-06  1.36529661e-06  3.17736571e-06  6.23318936e-10
  -4.89965147e-07  4.92475394e-07]
 [ 7.76814009e-02  7.71700387e-02  6.23318936e-10  5.46273686e-01
   1.27167783e-02  1.27711668e-02]
 [ 7.51482709e-04  7.44525977e-04 -4.89965147e-07  1.27167783e-02
   8.29124518e-04  8.26520517e-04]
 [ 7.58574521e-04  7.52563748e-04  4.92475394e-07  1.27711668e-02
   8.26520517e-04  8.25558551e-04]]
(6, 6)


Then we perform SMM using optimal weighting matrix. The estimated parameters and minimized criterion function values are as below. The estimated parameters do not change much and minimized criterion function value becomes larger.

In [64]:
params_init2 = np.array([alpha_SMM1, mu_SMM1, sig_SMM1, p_SMM1])

smm_args2 = (datavals, unif_vals, T, S, W_hat2)
results2 = opt.minimize(criterion, params_init2, args=(smm_args2),
                          method='L-BFGS-B',
                          bounds=((1e-10, 1-1e-10), (1e-10, None), (1e-10, None),(-1+1e-10, 1-1e-10)))
alpha_SMM2, mu_SMM2, sig_SMM2, p_SMM2 = results2.x
print('alpha_SMM2=', alpha_SMM2, ' mu_SMM2=', mu_SMM2, ' sig_SMM2=', sig_SMM2, ' p_SMM2=', p_SMM2)
print(results2)

alpha_SMM2= 0.4207519656565238  mu_SMM2= 9.934591487172444  sig_SMM2= 0.0873344718660021  p_SMM2= 0.9227031553066365
      fun: 0.7158887680866852
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 6.59252319e-02, -8.10462808e-06, -2.25851560e-03, -6.88504809e-04])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 195
      nit: 33
   status: 0
  success: True
        x: array([0.42075197, 9.93459149, 0.08733447, 0.92270316])


The difference vector values at the optimum below also becomes larger.

In [69]:
err_vec(datavals, unif_vals, alpha_SMM2, 0.99, mu_SMM2, sig_SMM2, p_SMM2, T, S, False)

array([-0.00811926, -0.01073198, -0.0012743 , -0.0240484 ,  0.00128095,
        0.00071323])

Then, what about standard error for each estimated parameter? Below we show standard errors for each paramter using optimized weighting matrix. We can see that standard errors are much smaller than parameters using identity weighting matrix. Thus, these etimated parameters are much efficient. 

In [65]:
d_err2 = Jac_err(datavals, unif_vals, alpha_SMM2, mu_SMM2, sig_SMM2, p_SMM2, False)
print(d_err2)
SigHat2 = (1 / S) * lin.inv(d_err2.T @ W_hat2 @ d_err2)
print(SigHat2)
print('Std. err. alpha_hat=', np.sqrt(SigHat2[0, 0]))
print('Std. err. mu_hat=', np.sqrt(SigHat2[1, 1]))
print('Std. err. sig_hat=', np.sqrt(SigHat2[2, 2]))
print('Std. err. p_hat=', np.sqrt(SigHat2[3, 3]))

[[ 2.67875848e+01  1.70027484e+00  1.42028810e+00  7.38929363e-01]
 [ 3.04336685e+01  1.67853231e+00  1.40067115e+00  7.29536758e-01]
 [-1.69462513e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 5.97553588e+01  3.37263099e+00  2.68851319e+01  1.18325819e+01]
 [ 1.29355719e-01  2.30200838e-04 -8.73739082e-02  4.24393475e-01]
 [ 2.18349910e-01  5.08610602e-03 -8.45958595e-02  4.30321310e-01]]
[[ 3.16561009e-10  4.01849212e-10  1.21475466e-09 -2.39653822e-09]
 [ 4.01849212e-10  8.37616235e-06  1.13566614e-07 -9.37697395e-07]
 [ 1.21475466e-09  1.13566614e-07  4.11771239e-07 -6.17819778e-07]
 [-2.39653822e-09 -9.37697395e-07 -6.17819778e-07  3.86425086e-06]]
Std. err. alpha_hat= 1.779216144204821e-05
Std. err. mu_hat= 0.0028941600428499404
Std. err. sig_hat= 0.0006416940382451474
Std. err. p_hat= 0.0019657697891615374
